In [7]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers

from sklearn.metrics import roc_auc_score, roc_curve

In [8]:
def mmd(x1, x2, beta):
    x1x1 = gaussian_kernel(x1, x1, beta)
    x1x2 = gaussian_kernel(x1, x2, beta)
    x2x2 = gaussian_kernel(x2, x2, beta)
    return tf.reduce_mean(x1x1) - 2 * tf.reduce_mean(x1x2) + tf.reduce_mean(x2x2)

def gaussian_kernel(x1, x2, beta = 1.0):
    k = tf.reduce_sum((x1[:, tf.newaxis, :] - x2[tf.newaxis, :, :]) ** 2, axis=-1)
    return tf.exp(-1 * beta * k)

In [15]:
class DNMC(Model):
    
    def __init__(self,
                 phi_layer_sizes=[100,], psi_layer_sizes=[100,], omega_layer_sizes=[100,],
                 e_layer_sizes=[100,], t_layer_sizes=[100,], c_layer_sizes=[100,],
                 n_bins=50,
                 activation='relu',
                 ld=1e-3, lr=1e-3):
        
        super(DNMC, self).__init__()
        
        self.ld = ld
        self.lr = lr
        self.activation=activation
        
        self.phi_model = Sequential([self.dense(ls) for ls in phi_layer_sizes])
        self.psi_model = Sequential([self.dense(ls) for ls in psi_layer_sizes])
        self.omega_model = Sequential([self.dense(ls) for ls in omega_layer_sizes])
        
        self.e_model = Sequential([self.dense(ls) for ls in e_layer_sizes] + [Dense(1, activation='sigmoid')])
        self.t_model = Sequential([self.dense(ls) for ls in t_layer_sizes] + [Dense(n_bins, activation='softmax')])
        
        
    def dense(self, layer_size):
        
        layer = Dense(
            layer_size,
            activation=self.activation,
            kernel_regularizer=regularizers.l2(self.lr),
            bias_regularizer=regularizers.l2(self.lr))
        
        return(layer)
        
    
    def forward_pass(self, x):
        
        self.phi = self.phi_model(x)
        self.psi = self.psi_model(x)
        self.omega = self.omega_model(x)
        
        self.e = self.e_model(tf.concat([self.phi, self.psi], axis=-1))
        self.t = self.t_model(tf.concat([self.psi, self.omega], axis=-1))
        
        return self.e, self.t
    
    
    def call(self, x):
        return self.forward_pass(x)

    
    def loss(self, x, t, s):
        return self.nll(x, t, s) + self.ld * self._mmd(x, s)
    
    
    def nll(self, x, t, s):
        pass
    
    
    def _mmd(self, x, s):
        return mmd(x[s == 1], x[s == 0])


In [16]:
m = DNMC()

In [17]:
m(np.random.rand(500, 10))

(<tf.Tensor: shape=(500, 1), dtype=float32, numpy=
 array([[0.4808621 ],
        [0.41283065],
        [0.4778645 ],
        [0.5065267 ],
        [0.42270193],
        [0.45947406],
        [0.4116763 ],
        [0.48572376],
        [0.47838238],
        [0.444885  ],
        [0.5084137 ],
        [0.40363657],
        [0.49509874],
        [0.48325527],
        [0.44744757],
        [0.43191797],
        [0.4637756 ],
        [0.44094288],
        [0.4375015 ],
        [0.45862827],
        [0.46010488],
        [0.44419172],
        [0.44589296],
        [0.4354164 ],
        [0.42678818],
        [0.5076218 ],
        [0.46617603],
        [0.44706565],
        [0.45680356],
        [0.43134636],
        [0.45880175],
        [0.43535087],
        [0.46386415],
        [0.4820342 ],
        [0.46292704],
        [0.47683358],
        [0.45488617],
        [0.48314965],
        [0.44111115],
        [0.4619184 ],
        [0.4562267 ],
        [0.418679  ],
        [0.45644775],
   